# Pythonic examples


## Proverb example

In [4]:
import guidance

# define a guidance program that adapts a proverb
@guidance
def program(lm, proverb, book, chapter, verse):
    return lm(f"""Tweak this proverb to apply to model instructions instead.

{proverb}
- {book} {chapter}:{verse}

UPDATED
Where there is no guidance""").gen("rewrite", stop="\n-")("""
- GPT """).select("chapter", options=[9, 9, 11], suffix=":").gen("verse")

lm = guidance.models.OpenAI("text-davinci-003", caching=False)

# execute the program on a specific proverb
out = program(lm,
    proverb="Where there is no guidance, a people falls,\nbut in an abundance of counselors there is safety.",
    book="Proverbs",
    chapter=11,
    verse=14
)


In [3]:
out["verse"]

'14'

# Chat dialog

In [2]:
import guidance

from guidance.library import gen, select, user, assistant, system

# gpt4 = guidance.models.OpenAI("gpt-4", caching=False)
gpt4 = guidance.models.OpenAI("gpt-3.5-turbo", caching=False)

def experts(lm, query):
    lm = lm.system("You are a helpful and terse assistant.")

    with lm.user() as lm:
        lm += f"""I want a response to the following question:
{query}
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet."""

    with lm.assistant() as lm:
        lm = lm.gen('expert_names', temperature=0, max_tokens=30)

    lm = lm.user("Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.")

    with lm.assistant() as lm:
        lm = lm.gen('answer', temperature=0, max_tokens=50)

    return lm

ans = experts(gpt4, query='How can I be more productive?')

## JSON example

In [3]:
import guidance

# we can pre-define valid option sets
valid_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow"]

# define the prompt

def character_maker(lm, id, description, valid_weapons):
    lm = lm(f'''The following is a character profile for an RPG game in JSON format.
```json
{{
    "id": "{id}",
    "description": "{description}",
    "name": "''').gen('name', stop='"', suffix='''"
    "age": ''').gen('age', pattern='[0-9]+', stop='"', suffix=''',
    "armor": "''').select('armor', options=["leather", "chainmail", "plate"], suffix='''",
    "weapon": "''').select('armor', options=valid_weapons, suffix='''",
    "class": "''').gen('class', stop='"', suffix='''",
    "mantra": "''').gen('mantra', temperature=0.7, stop='"', suffix='''",
    "strength": ''').gen('strength', pattern='[0-9]+', suffix=''',
    "items": [''')
    
    for i in range(5):
        if i > 0:
            lm += ', '
        lm = lm('"').gen('item', temperature=0.7, suffix='"')
    lm += ''']
}```'''

llama = guidance.models.Transformers("gpt2")

# generate a character
character_maker(llama,
    id="e1f491f7-7ab8-4dac-8c20-c92b5e7d883d",
    description="A quick and nimble fighter.",
    valid_weapons=valid_weapons
)


In [2]:
# we use LLaMA here, but any GPT-style model will do
import guidance

@guidance
def quoted_list(lm, count):
    lm += '['
    for i in range(count):
        if i > 0:
            lm += ', '
        lm = lm('"').gen('item', temperature=0.7, stop='"')('"')
    lm += ']'
    return lm

# we can pre-define valid option sets
valid_weapons = ["sword", "axe", "mace", "spear", "bow", "crossbow"]

# define the prompt
@guidance
def character_maker(lm, id, description, valid_weapons):
    lm = lm(f'''The following is a character profile for an RPG game in JSON format.
```json
{{
    "id": "{id}",
    "description": "{description}",
    "name": "''').gen('name', stop='"')('''",
    "age": ''').gen('age', pattern='[0-9]+', stop=',')(''',
    "armor": "''').select('armor', options=["leather", "chainmail", "plate"], suffix=''',
    "weapon": "''').select('armor', options=valid_weapons, suffix='''",
    "class": "''').gen('class', stop='"')('''",
    "mantra": "''').gen('mantra', temperature=0.7, stop='"')('''",
    "strength": ''').gen('strength', pattern='[0-9]+', stop=',')(''',
    "items": ''').quoted_list(count=5)('''
}```''')
    return lm

lm = guidance.models.Transformers("gpt2", caching=True)

# generate a character
lm.character_maker(
    id="e1f491f7-7ab8-4dac-8c20-c92b5e7d883d",
    description="A quick and nimble fighter.",
    valid_weapons=valid_weapons
)


## Token healing

In [1]:
import guidance

# we use StableLM as an open example, but these issues impact all models to varying degrees
lm = guidance.models.Transformers("stabilityai/stablelm-base-alpha-3b", device=0)

# we turn token healing off so that guidance acts like a normal prompting library
lm('The link is <a href="http:').gen(max_tokens=10, token_healing=False)

In [2]:
lm('The link is <a href="http:').gen(max_tokens=10)

## Rich output structure

In [1]:
import guidance
                                                      
# set the default language model used to execute guidance programs
lm = guidance.models.OpenAI("text-davinci-003") 

# define the few shot examples
examples = [
    {'input': 'I wrote about shakespeare',
    'entities': [{'entity': 'I', 'time': 'present'}, {'entity': 'Shakespeare', 'time': '16th century'}],
    'reasoning': 'I can write about Shakespeare because he lived in the past with respect to me.',
    'answer': 'No'},
    {'input': 'Shakespeare wrote about me',
    'entities': [{'entity': 'Shakespeare', 'time': '16th century'}, {'entity': 'I', 'time': 'present'}],
    'reasoning': 'Shakespeare cannot have written about me, because he died before I was born',
    'answer': 'Yes'}
]

# define the guidance program
def structure_program(lm, examples, input):
    lm += '''Given a sentence tell me whether it contains an anachronism (i.e. whether it could have happened or not based on the time periods associated with the entities).
----'''

    # display the few-shot examples
    for example in examples:
        lm += f'''
Sentence: {example['input']}
Entities and dates:'''
        for entity in example['entities']:
            lm += f'''
{entity['entity']}: {entity['time']}'''
        lm += f'''
Reasoning: {example['reasoning']}
Anachronism: {example['answer']}
---'''


    # place the real question at the end
    lm = lm(f'''
Sentence: {input}
Entities and dates:
''').gen("entities", max_tokens=100, suffix='''
Reasoning:''').gen("reasoning", suffix='''
Anachronism:''').select("answer", options=[" Yes", " No"])

# execute the program
out = structure_program(lm,
    examples=examples,
    input='The T-rex bit my dog'
)

## Function calling

In [3]:
import guidance

# define a tool we would like the model to use
import json
def get_current_weather(location, unit="fahrenheit"):
    """ Get the current weather in a given location.
    
    Parameters
    ----------
    location : string
        The city and state, e.g. San Francisco, CA
    unit : "celsius" or "fahrenheit"
    """
    weather_info = {
        "location": location,
        "temperature": "71",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

# define the chat model we want to use (must be a recent model supporting function calls)
lm = guidance.models.OpenAIChat("gpt-3.5-turbo-0613", caching=False)

# define a guidance program that uses tools
def program(lm, location, functions):
    with lm.system() as lm:
        lm += "You are a helpful assistant."
        lm = lm.tool_def(functions=functions)

    lm = lm.user(f"Get the current weather in {location}.")

    for i in range(10):
        with lm.assistant() as lm:
            lm = lm.gen('answer', max_tokens=50, function_call="auto")
        ans = lm["answer"]
        
        if not callable(ans):
            break
        
        lm = lm.function(ans(), name=ans.__name__)

executed_program = program(lm, "New York City", functions=[
    {
        "name": "get_current_weather",
        "callable": get_current_weather,
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        }
    }
])